# A100 & A200 Watercooling Test - Dual Pressure & Temperature Monitoring

This notebook provides control and monitoring capabilities for A100 and A200 pressure sensors with integrated chiller temperature monitoring using the current hardware versions.

## Features:
- Monitoring for 2 pressure sensors (A100 & A200) via TPG366 controller (current version)
- Chiller (Lauda) temperature monitoring and control
- Shared logging system using loguru
- Cancelable continuous monitoring
- Real-time data visualization including pressure and temperature
- Simplified and optimized for current hardware

## Test Configuration:
- A100 Pressure (Channel 1)
- A200 Pressure (Channel 2) 
- Water Temperature (Chiller connected to Pump Locker and heat exchanger)
- Starting both pressure monitoring and temperature logging simultaneously

## 1. Import Required Libraries and Setup

In [1]:
import sys
import threading
import time
from datetime import datetime
import asyncio
from typing import Dict, Optional, Any
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import display, clear_output
import ipywidgets as widgets
import pickle

from loguru import logger
import os
from pathlib import Path

# Add path to src modules
sys.path.append(os.path.join(os.getcwd(), '..', '..', 'src'))

# Import current device modules
from devices.pfeiffer.tpg366.tpg366 import TPG366
from devices.chiller.chiller import Chiller

## 2. Setup Shared Logging System

In [2]:
# Get the repository root and create shared logs directory
repo_root = Path(os.getcwd()).parent.parent
log_dir = repo_root / "debugging" / "logs"
log_dir.mkdir(parents=True, exist_ok=True)

# Create shared log file for all devices
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
shared_log_file = log_dir / f"009_A100_watercooling_test_withN2_{timestamp}.log"

# Configure shared logger
logger.remove()  # Remove default logger

# Add console logger with INFO level
logger.add(sys.stderr, level="INFO", format="{time:YYYY-MM-DD HH:mm:ss} | {level} | {message}")

# Add shared file logger with DEBUG level
logger.add(
    str(shared_log_file),
    level="DEBUG",
    format="{time:YYYY-MM-DD HH:mm:ss.SSS} | {level} | {name}:{function}:{line} | {message}",
    rotation="1 day",
    retention="30 days",
    compression="zip"
)

logger.info("A200 Watercooling test system initialized with shared logging")
print(f"Repository root: {repo_root}")
print(f"Shared logs will be saved to: {shared_log_file}")

2025-09-16 14:59:23 | INFO | A200 Watercooling test system initialized with shared logging


Repository root: C:\Users\ESIBDlab\PycharmProjects\esibd_bs
Shared logs will be saved to: C:\Users\ESIBDlab\PycharmProjects\esibd_bs\debugging\logs\009_A100_watercooling_test_withN2_20250916_145923.log


## 3. Dual Pressure & Temperature Controller Class

In [3]:
class DualPressureTemperatureController:
    """Controller class for managing dual pressure sensors (A100/A200) and chiller temperature monitoring"""
    
    def __init__(self):
        self.pressure_controller: Optional[TPG366] = None
        self.chiller: Optional[Chiller] = None
        
        # Monitoring control
        self.monitoring_active = False
        self.monitoring_thread = None
        self.monitoring_stop_event = threading.Event()
        
        # Data storage
        self.monitoring_data = []
        
        # Shared thread lock for all devices
        self.shared_lock = threading.Lock()
        
        logger.info("Dual Pressure & Temperature Controller initialized")
    
    def connect_pressure_controller(self, com_port: str, address: int):
        """Connect to TPG366 pressure controller with shared logger"""
        try:
            logger.info(f"Connecting to TPG366 pressure controller on {com_port}, address {address}")
            self.pressure_controller = TPG366(
                device_id="tpg366_dual_pressure",
                port=com_port,
                device_address=address,
                logger=logger,  # Pass shared logger
                thread_lock=self.shared_lock  # Use shared lock
            )
            self.pressure_controller.connect()
            
            logger.success("TPG366 pressure controller connected successfully")
            return True
            
        except Exception as e:
            logger.error(f"Failed to connect TPG366 pressure controller: {e}")
            return False
    
    def connect_chiller(self, com_port: str):
        """Connect to Chiller (Lauda) with shared logger"""
        try:
            logger.info(f"Connecting to Chiller on {com_port}")
            self.chiller = Chiller(
                device_id="lauda_chiller",
                port=com_port,
                logger=logger,  # Pass shared logger
                thread_lock=self.shared_lock  # Use shared lock
            )
            self.chiller.connect()
            
            logger.success("Chiller connected successfully")
            return True
            
        except Exception as e:
            logger.error(f"Failed to connect Chiller: {e}")
            return False
    
    def disconnect_all(self):
        """Safely disconnect all devices"""
        logger.info("Disconnecting all devices...")
        
        # Stop monitoring first
        self.stop_monitoring()
        
        # Disconnect pressure controller
        if self.pressure_controller:
            try:
                self.pressure_controller.disconnect()
                logger.info("TPG366 pressure controller disconnected")
            except Exception as e:
                logger.error(f"Error disconnecting TPG366 pressure controller: {e}")
        
        # Disconnect chiller
        if self.chiller:
            try:
                self.chiller.disconnect()
                logger.info("Chiller disconnected")
            except Exception as e:
                logger.error(f"Error disconnecting Chiller: {e}")
        
        logger.success("All devices disconnected")

# Create global controller instance
controller = DualPressureTemperatureController()

2025-09-16 14:59:32 | INFO | Dual Pressure & Temperature Controller initialized


## 4. Device Monitoring Functions

In [4]:
def monitor_tpg366_pressure(pressure_controller: TPG366, sensor_id: str, channel: int) -> Dict[str, Any]:
    """
    Monitor pressure from TPG366 sensor channel.
    
    Parameters:
    - pressure_controller: TPG366 pressure controller instance
    - sensor_id: Identifier string for the sensor (A100/A200)
    - channel: Sensor channel to read from (1-6)
    
    Returns:
    - Dictionary containing pressure data
    """
    
    try:
        # Get actual pressure reading using current API
        pressure_value = pressure_controller.read_pressure_value(channel)
        
        pressure_data = {
            'timestamp': datetime.now(),
            'sensor_id': sensor_id,
            'channel': channel,
            'pressure_value': pressure_value,
            'status': 'OK'
        }
        
        # Log single value with consistent format
        logger.info(f"Pressure monitoring, {sensor_id}_Pressure = {pressure_value} mbar")
        return pressure_data
        
    except Exception as e:
        logger.error(f"Error reading pressure from {sensor_id}: {e}")
        return {
            'timestamp': datetime.now(),
            'sensor_id': sensor_id,
            'channel': channel,
            'pressure_value': None,
            'error': str(e),
            'status': 'ERROR'
        }

def monitor_chiller_temperature(chiller: Chiller) -> Dict[str, Any]:
    """
    Monitor temperature from Chiller.
    
    Parameters:
    - chiller: Chiller instance
    
    Returns:
    - Dictionary containing temperature data
    """
    
    try:
        # Get actual temperature reading
        current_temp = chiller.read_temp()
        set_temp = chiller.read_set_temp()
        
        temp_data = {
            'timestamp': datetime.now(),
            'current_temperature': current_temp,
            'set_temperature': set_temp,
            'status': 'OK'
        }
        
        # Log each temperature value separately with consistent format
        logger.info(f"Temperature monitoring, Chiller_Current_Temp = {current_temp} °C")
        logger.info(f"Temperature monitoring, Chiller_Set_Temp = {set_temp} °C")
        return temp_data
        
    except Exception as e:
        logger.error(f"Error reading temperature from Chiller: {e}")
        return {
            'timestamp': datetime.now(),
            'current_temperature': None,
            'set_temperature': None,
            'error': str(e),
            'status': 'ERROR'
        }

print("Monitoring functions defined successfully")

Monitoring functions defined successfully


## 5. Continuous Monitoring System

In [5]:
def continuous_monitoring_worker(controller: DualPressureTemperatureController, interval: float = 5.0):
    """
    Worker function for continuous monitoring that runs in a separate thread.
    """
    logger.info(f"Starting continuous monitoring with {interval}s interval")
    
    while not controller.monitoring_stop_event.is_set():
        try:
            monitoring_cycle_data = {
                'cycle_timestamp': datetime.now(),
                'a100_data': None,
                'a200_data': None,
                'chiller_data': None
            }

            # Monitor pressure sensors if controller is connected
            if controller.pressure_controller:
                # Read from A100 sensor (Channel 1)
                monitoring_cycle_data['a100_data'] = monitor_tpg366_pressure(
                    controller.pressure_controller, "A100", channel=1
                )
                
                # Read from A200 sensor (Channel 2)
                monitoring_cycle_data['a200_data'] = monitor_tpg366_pressure(
                    controller.pressure_controller, "A200", channel=2
                )
            
            # Monitor chiller temperature if connected
            if controller.chiller:
                monitoring_cycle_data['chiller_data'] = monitor_chiller_temperature(
                    controller.chiller
                )
            
            # Store data
            controller.monitoring_data.append(monitoring_cycle_data)
            
            # Log monitoring cycle completion (individual values already logged above)
            logger.debug(f"Monitoring cycle completed at {monitoring_cycle_data['cycle_timestamp']}")
            
            # Wait for next cycle or stop signal
            if controller.monitoring_stop_event.wait(timeout=interval):
                break  # Stop event was set
                
        except Exception as e:
            logger.error(f"Error in monitoring cycle: {e}")
            # Continue monitoring even if one cycle fails
            time.sleep(interval)
    
    logger.info("Continuous monitoring stopped")

def start_monitoring(controller: DualPressureTemperatureController, interval: float = 5.0):
    """Start continuous monitoring in a separate thread."""
    if controller.monitoring_active:
        logger.warning("Monitoring is already active")
        return False
    
    # Reset stop event and clear old data
    controller.monitoring_stop_event.clear()
    controller.monitoring_data.clear()
    
    # Start monitoring thread
    controller.monitoring_thread = threading.Thread(
        target=continuous_monitoring_worker,
        args=(controller, interval),
        daemon=True
    )
    
    controller.monitoring_thread.start()
    controller.monitoring_active = True
    
    logger.success(f"Continuous monitoring started with {interval}s interval")
    return True

def stop_monitoring(controller: DualPressureTemperatureController):
    """Stop continuous monitoring gracefully."""
    if not controller.monitoring_active:
        logger.info("Monitoring is not currently active")
        return
    
    logger.info("Stopping continuous monitoring...")
    
    # Signal the monitoring thread to stop
    controller.monitoring_stop_event.set()
    
    # Wait for thread to finish (with timeout)
    if controller.monitoring_thread and controller.monitoring_thread.is_alive():
        controller.monitoring_thread.join(timeout=10.0)
        
        if controller.monitoring_thread.is_alive():
            logger.warning("Monitoring thread did not stop gracefully")
        else:
            logger.success("Monitoring thread stopped successfully")
    
    controller.monitoring_active = False
    controller.monitoring_thread = None

# Add methods to controller class
controller.start_monitoring = lambda interval=5.0: start_monitoring(controller, interval)
controller.stop_monitoring = lambda: stop_monitoring(controller)

print("Continuous monitoring system ready")

Continuous monitoring system ready


## 6. Device Connection Configuration

In [6]:
# Configuration for your devices
# Modify these values according to your setup

# TPG366 pressure controller
PRESSURE_COM = 'COM22'  # TPG366 pressure controller COM port
PRESSURE_ADDR = 10      # TPG366 standard address

# Chiller (Lauda) configuration
CHILLER_COM = 'COM39'    # Chiller COM port (modify as needed)

print("Device configuration loaded:")
print(f"TPG366 Pressure Controller: {PRESSURE_COM}, Address {PRESSURE_ADDR}")
print(f"  -> A100 Sensor: Channel 1")
print(f"  -> A200 Sensor: Channel 2")
print(f"Lauda Chiller: {CHILLER_COM}")
print(f"  -> Water temperature monitoring and control")

Device configuration loaded:
TPG366 Pressure Controller: COM22, Address 10
  -> A100 Sensor: Channel 1
  -> A200 Sensor: Channel 2
Lauda Chiller: COM39
  -> Water temperature monitoring and control


In [7]:
# Connect to TPG366 pressure controller
pressure_connection = controller.connect_pressure_controller(PRESSURE_COM, PRESSURE_ADDR)

if pressure_connection:
    print("✅ TPG366 pressure controller connected successfully")
    print("A100 and A200 sensors accessible via channels 1 and 2")
else:
    print("❌ Failed to connect to TPG366 pressure controller")

# Connect to Chiller
chiller_connection = controller.connect_chiller(CHILLER_COM)

if chiller_connection:
    print("✅ Lauda Chiller connected successfully")
    print("Water temperature monitoring enabled")
else:
    print("❌ Failed to connect to Lauda Chiller")

2025-09-16 14:59:38 | INFO | Connecting to TPG366 pressure controller on COM22, address 10
2025-09-16 14:59:38 | INFO | Connecting to Pfeiffer device tpg366_dual_pressure on COM22
2025-09-16 14:59:38 | INFO | Successfully connected to device at address 10
2025-09-16 14:59:38 | SUCCESS | TPG366 pressure controller connected successfully
2025-09-16 14:59:38 | INFO | Connecting to Chiller on COM39
2025-09-16 14:59:38 | INFO | Using external logger for device 'lauda_chiller'
2025-09-16 14:59:38 | INFO | Connecting to chiller lauda_chiller on COM39
2025-09-16 14:59:38 | SUCCESS | Chiller connected successfully


✅ TPG366 pressure controller connected successfully
A100 and A200 sensors accessible via channels 1 and 2
✅ Lauda Chiller connected successfully
Water temperature monitoring enabled


## 7. Device Testing and Status

In [8]:
# Test individual sensor readings (A100 and A200)
if controller.pressure_controller:
    try:
        print("Testing A100 and A200 Pressure Sensors:")
        
        # Test A100 (Channel 1)
        a100_pressure = controller.pressure_controller.read_pressure_value(1)
        print(f"  A100 Pressure (Channel 1): {a100_pressure}")
        
        # Test A200 (Channel 2) 
        a200_pressure = controller.pressure_controller.read_pressure_value(2)
        print(f"  A200 Pressure (Channel 2): {a200_pressure}")
        
    except Exception as e:
        logger.error(f"Error testing pressure sensors: {e}")
        print(f"  ❌ Error testing pressure sensors: {e}")
else:
    print("TPG366 pressure controller not connected")

Testing A100 and A200 Pressure Sensors:
  A100 Pressure (Channel 1): 1014.9999999999999
  A200 Pressure (Channel 2): 1008.9999999999999


In [9]:
# Test Chiller temperature readings
if controller.chiller:
    try:
        print("Testing Lauda Chiller:")
        
        # Get current and set temperatures
        current_temp = controller.chiller.read_temp()
        set_temp = controller.chiller.read_set_temp()
        print(f"  Current Temperature: {current_temp}°C")
        print(f"  Set Temperature: {set_temp}°C")
        
        # Get device status
        status = controller.chiller.read_status()
        running = controller.chiller.read_running()
        print(f"  Device Status: {status}")
        print(f"  Running State: {running}")
        
        print("\n  Chiller tests completed successfully ✅")
        
    except Exception as e:
        logger.error(f"Error testing Chiller: {e}")
        print(f"  ❌ Error testing Chiller: {e}")
else:
    print("Chiller not connected")

Testing Lauda Chiller:
  Current Temperature: 17.31°C
  Set Temperature: 16.0°C
  Device Status: OK
  Running State: DEVICE RUNNING

  Chiller tests completed successfully ✅


## 8. Monitoring Control Interface

In [10]:
# Create interactive controls for monitoring
start_button = widgets.Button(description="Start Monitoring", button_style='success')
stop_button = widgets.Button(description="Stop Monitoring", button_style='danger')
interval_slider = widgets.FloatSlider(
    value=2.0,
    min=0.1,
    max=2.0,
    step=0.1,
    description='Interval (s):',
    style={'description_width': 'initial'}
)
status_output = widgets.Output()

def on_start_clicked(b):
    with status_output:
        clear_output(wait=True)
        success = controller.start_monitoring(interval_slider.value)
        if success:
            print(f"✅ Monitoring started with {interval_slider.value}s interval")
            print("Monitoring: A100 pressure, A200 pressure, and chiller temperature")
        else:
            print("❌ Failed to start monitoring (already running?)")

def on_stop_clicked(b):
    with status_output:
        clear_output(wait=True)
        controller.stop_monitoring()
        print("🛑 Monitoring stopped")

start_button.on_click(on_start_clicked)
stop_button.on_click(on_stop_clicked)

# Display controls
controls = widgets.VBox([
    widgets.Label("A100/A200 & Chiller Monitoring Controls:"),
    interval_slider,
    widgets.HBox([start_button, stop_button]),
    status_output
])

display(controls)

In [50]:
%matplotlib inline

## 9. Data Visualization

### 9.1 Static Plotting

In [17]:
def plot_monitoring_data():
    """Plot the collected monitoring data for dual pressure sensors and chiller temperature"""
    if not controller.monitoring_data:
        print("No monitoring data available. Start monitoring first.")
        return
    
    # Convert data to lists for easier plotting
    timestamps = []
    a100_pressure = []
    a200_pressure = []
    chiller_current_temp = []
    #chiller_set_temp = []

    for cycle in controller.monitoring_data:
        timestamps.append(cycle['cycle_timestamp'])
        
        # Extract pressure data
        a100_pressure.append(cycle['a100_data']['pressure_value'] if cycle['a100_data'] and cycle['a100_data'].get('pressure_value') else None)
        a200_pressure.append(cycle['a200_data']['pressure_value'] if cycle['a200_data'] and cycle['a200_data'].get('pressure_value') else None)
        
        # Extract temperature data
        chiller_current_temp.append(cycle['chiller_data']['current_temperature'] if cycle['chiller_data'] and cycle['chiller_data'].get('current_temperature') else None)
        #chiller_set_temp.append(cycle['chiller_data']['set_temperature'] if cycle['chiller_data'] and cycle['chiller_data'].get('set_temperature') else None)

    # Create subplots: 2 rows, 1 column
    fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(8, 6), sharex=True)
    
    # Plot 1: Pressure (with logarithmic scale for vacuum)
    ax1.plot(timestamps, a100_pressure, 'b-', label='A100 Pressure', marker='o', markersize = 1)
    #ax1.plot(timestamps, a200_pressure, 'r-', label='A200 Pressure', marker='^', markersize = 1)
    ax1.set_yscale('log')
    ax1.set_ylabel('Pressure (log scale)')
    ax1.set_title('A100 & A200 Pressure Over Time')
    ax1.legend()
    ax1.grid(True)
    ax1.tick_params(axis='x', rotation=45)
    
    # Plot 2: Temperature
    ax2.plot(timestamps, chiller_current_temp, 'g-', label='Current Temperature', marker='s', markersize = 1)
    #ax2.plot(timestamps, chiller_set_temp, 'g--', label='Set Temperature', marker='d', alpha=0.7)
    ax2.set_ylabel('Temperature // degC')
    ax2.set_xlabel('Time // HH:MM:SS')
    ax2.set_title('Chiller Temperature Over Time')
    ax2.legend()
    ax2.grid(True)
    ax2.tick_params(axis='x', rotation=45)
    import matplotlib.dates as mdates
    ax2.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
    ax2.xaxis.set_major_locator(mdates.MinuteLocator(interval=20))  # Adjust interval as needed
    
    plt.tight_layout()
    plt.show()
    
    # Print statistics
    print(f"Plotted {len(controller.monitoring_data)} data points")
    valid_a100 = [p for p in a100_pressure if p is not None]
    valid_a200 = [p for p in a200_pressure if p is not None]
    valid_temp = [t for t in chiller_current_temp if t is not None]

    if valid_a100:
        print(f"A100 pressure range: {min(valid_a100):.2e} to {max(valid_a100):.2e}")
    if valid_a200:
        print(f"A200 pressure range: {min(valid_a200):.2e} to {max(valid_a200):.2e}")
    if valid_temp:
        print(f"Chiller temperature range: {min(valid_temp):.2f}°C to {max(valid_temp):.2f}°C")

def get_monitoring_summary():
    """Get a summary of current monitoring status"""
    print(f"Monitoring Status: {'🟢 Active' if controller.monitoring_active else '🔴 Inactive'}")
    print(f"Data Points Collected: {len(controller.monitoring_data)}")
    print(f"TPG366 Controller Connected: {'✅' if controller.pressure_controller else '❌'}")
    print(f"Chiller Connected: {'✅' if controller.chiller else '❌'}")
    
    if controller.monitoring_data:
        latest = controller.monitoring_data[-1]
        print(f"\nLatest Data Point: {latest['cycle_timestamp']}")
        
        # Show latest pressure readings
        if latest['a100_data']:
            a100_p = latest['a100_data'].get('pressure_value')
            print(f"  A100 Pressure: {a100_p}")
        if latest['a200_data']:
            a200_p = latest['a200_data'].get('pressure_value')
            print(f"  A200 Pressure: {a200_p}")
        
        # Show latest temperature readings
        if latest['chiller_data']:
            curr_temp = latest['chiller_data'].get('current_temperature')
            set_temp = latest['chiller_data'].get('set_temperature')
            print(f"  Chiller Temperature: {curr_temp}°C (set: {set_temp}°C)")

print("Data visualization functions ready")

Data visualization functions ready


In [ ]:
# Plot monitoring data (run this cell periodically to see updated plots)
plot_monitoring_data()

In [ ]:
# Get current monitoring summary
get_monitoring_summary()

### 9.2 Live Plotting

In [19]:
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import numpy as np

class DualPressureTemperaturePlotter:
    def __init__(self, controller, update_interval=1000):
        """
        Initialize live monitoring plot for dual pressure and temperature
        Args:
            controller: The controller object with monitoring_data
            update_interval: Update interval in milliseconds (default: 1000ms = 1 second)
        """
        self.controller = controller
        self.update_interval = update_interval

        # Initialize data lists
        self.timestamps = []
        self.a100_pressure = []
        self.a200_pressure = []
        self.chiller_current_temp = []

        # Create figure and subplots (2 rows, 1 column)
        self.fig, (self.ax1, self.ax2) = plt.subplots(2, 1, figsize=(10, 8))

        self.temp_log = []
        self.log_file_size = 0

    def update_plot(self, frame):
        # Get new data
        current_size = len(self.controller.monitoring_data)
        if current_size > self.log_file_size:
            new_entries = self.controller.monitoring_data[self.log_file_size:]
            self.temp_log.extend(new_entries)
            self.log_file_size = current_size

        # Process new entries
        for entry in self.temp_log:
            self.timestamps.append(entry['cycle_timestamp'])
            
            # Extract pressure data
            self.a100_pressure.append(entry['a100_data']['pressure_value'] if entry['a100_data'] and entry['a100_data'].get('pressure_value') else None)
            self.a200_pressure.append(entry['a200_data']['pressure_value'] if entry['a200_data'] and entry['a200_data'].get('pressure_value') else None)
            
            # Extract temperature data
            self.chiller_current_temp.append(entry['chiller_data']['current_temperature'] if entry['chiller_data'] and entry['chiller_data'].get('current_temperature') else None)

        # Clear temp_log after processing to avoid reprocessing
        self.temp_log = []

        # Clear only the axes, not the entire figure
        self.ax1.clear()
        self.ax2.clear()

        # Filter out None values for plotting
        valid_pressure_data = [(t, a100, a200) for t, a100, a200 in zip(self.timestamps, self.a100_pressure, self.a200_pressure)
                              if a100 is not None or a200 is not None]
        
        valid_temp_data = [(t, curr) for t, curr in zip(self.timestamps, self.chiller_current_temp)
                          if curr is not None]

        # Plot pressure data (ax1) - last 120 points only
        if valid_pressure_data:
            times_p, a100_vals, a200_vals = zip(*valid_pressure_data)

            # Plot A100 data (filter None values) - BLUE with circle marker
            a100_valid = [(t, p) for t, p in zip(times_p, a100_vals) if p is not None]
            if a100_valid:
                t_a100, p_a100 = zip(*a100_valid)
                if len(t_a100) > 120:
                    self.ax1.plot(t_a100[-119:], p_a100[-119:], 'b-', label='$P_{A100}$', marker='o')
                else:
                    self.ax1.plot(t_a100, p_a100, 'b-', label='$P_{A100}$', marker='o')

            # Plot A200 data (filter None values) - RED with triangle marker
            #a200_valid = [(t, p) for t, p in zip(times_p, a200_vals) if p is not None]
            #if a200_valid:
            #    t_a200, p_a200 = zip(*a200_valid)
            #    if len(t_a200) > 120:
            #        self.ax1.plot(t_a200[-119:], p_a200[-119:], 'r-', label='$P_{A200}$', marker='^')
            #    else:
            #        self.ax1.plot(t_a200, p_a200, 'r-', label='$P_{A200}$', marker='^')

        # Use logarithmic scale for pressure if values are very small (typical for vacuum)
        self.ax1.set_yscale('log')
        self.ax1.set_ylabel('Pressure (log scale)')
        self.ax1.set_title('A100 & A200 Pressure Over Time')
        self.ax1.legend()
        self.ax1.grid(True)

        # Plot temperature data (ax2) - last 120 points only
        if valid_temp_data:
            times_t, curr_temps = zip(*valid_temp_data)

            # Plot current temperature - GREEN with square marker
            curr_temp_valid = [(t, temp) for t, temp in zip(times_t, curr_temps) if temp is not None]
            if curr_temp_valid:
                t_curr, temp_curr = zip(*curr_temp_valid)
                if len(t_curr) > 120:
                    self.ax2.plot(t_curr[-119:], temp_curr[-119:], 'g-', label='$T_{Cooling}$', marker='s')
                else:
                    self.ax2.plot(t_curr, temp_curr, 'g-', label='$T_{Cooling}$', marker='s')


        self.ax2.set_ylabel('Temperature (degC)')
        self.ax2.set_xlabel('Time')
        self.ax2.set_title('Chiller: Water Temperature Over Time')
        self.ax2.legend()
        self.ax2.grid(True)

        # Rotate x-axis labels for better readability
        self.ax1.tick_params(axis='x', rotation=45)
        self.ax2.tick_params(axis='x', rotation=45)

        # Adjust layout
        self.fig.tight_layout()

    def start_live_plot(self):
        """Start the live plotting"""
        if not self.controller.monitoring_data:
            print("No monitoring data available. Start monitoring first.")
            return None, None

        print(f"Starting live plot with {self.update_interval/1000}s update interval...")
        print("Close the plot window to stop live plotting.")

        # Create animation
        self.ani = FuncAnimation(
            self.fig,
            self.update_plot,
            interval=self.update_interval,
            blit=False,  # Set to False for easier debugging
            cache_frame_data=False
        )

        # Show the plot
        plt.show()

        return self.timestamps, self.controller.monitoring_data

    def stop_live_plot(self):
        """Stop the live plotting"""
        if hasattr(self, 'ani'):
            self.ani.event_source.stop()
            print("Live plotting stopped.")
        else:
            print("No animation to stop.")

print("Live plotting class ready")

Live plotting class ready


In [12]:
%matplotlib notebook

In [ ]:
# Start live plotting
live_plot = DualPressureTemperaturePlotter(controller)
timestamps, data = live_plot.start_live_plot()

In [25]:
# Stop live plotting
live_plot.stop_live_plot()

Live plotting stopped.


## 10. Data Export and Cleanup

In [27]:
def export_monitoring_data(filename: str = None):
    """Export monitoring data to CSV file"""
    if not controller.monitoring_data:
        print("No data to export")
        return
    
    if filename is None:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"a100_a200_chiller_data_{timestamp}.csv"
    
    # Flatten the data for CSV export
    flattened_data = []
    for cycle in controller.monitoring_data:
        row = {'timestamp': cycle['cycle_timestamp']}
        
        # Add A100 data
        if cycle['a100_data']:
            for key, value in cycle['a100_data'].items():
                row[f'a100_{key}'] = value
        
        # Add A200 data
        if cycle['a200_data']:
            for key, value in cycle['a200_data'].items():
                row[f'a200_{key}'] = value
        
        # Add chiller data
        if cycle['chiller_data']:
            for key, value in cycle['chiller_data'].items():
                row[f'chiller_{key}'] = value
        
        flattened_data.append(row)
    
    df = pd.DataFrame(flattened_data)
    df.to_csv(filename, index=False)
    logger.info(f"Data exported to {filename}")
    print(f"✅ Data exported to {filename}")

def save_monitoring_data_pickle(filename: str = None):
    """Save monitoring data as pickle file"""
    if not controller.monitoring_data:
        print("No data to save")
        return
        
    if filename is None:
        timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
        filename = f"a100_a200_chiller_data_{timestamp}.pkl"
    
    with open(filename, 'wb') as f:
        pickle.dump(controller.monitoring_data, f)
    
    logger.info(f"Data saved to pickle file: {filename}")
    print(f"✅ Data saved to pickle file: {filename}")

def load_monitoring_data_pickle(filename: str):
    """Load monitoring data from pickle file"""
    try:
        with open(filename, 'rb') as f:
            loaded_data = pickle.load(f)
        
        controller.monitoring_data = loaded_data
        logger.info(f"Data loaded from pickle file: {filename}")
        print(f"✅ Data loaded from pickle file: {filename}")
        print(f"Loaded {len(loaded_data)} data points")
        
    except Exception as e:
        logger.error(f"Error loading data from {filename}: {e}")
        print(f"❌ Error loading data from {filename}: {e}")

print("Data export functions ready")

Data export functions ready


In [28]:
# Export current monitoring data
timestamp = datetime.now().strftime('%d-%m-%Y_%H%M')
export_monitoring_data(f"{timestamp}_a100_chiller_data_withN2.csv")

2025-09-16 16:39:21 | INFO | Data exported to 16-09-2025_1639_a100_chiller_data_withN2.csv


✅ Data exported to 16-09-2025_1639_a100_chiller_data_withN2.csv


In [29]:
# Save monitoring data as pickle
timestamp = datetime.now().strftime('%d-%m-%Y_%H%M')
save_monitoring_data_pickle(f"{timestamp}_a100_chiller_data_withN2.pkl")

2025-09-16 16:39:22 | INFO | Data saved to pickle file: 16-09-2025_1639_a100_chiller_data_withN2.pkl


✅ Data saved to pickle file: 16-09-2025_1639_a100_chiller_data_withN2.pkl


In [30]:
# Emergency disconnect all devices
controller.disconnect_all()

2025-09-16 16:39:23 | INFO | Disconnecting all devices...
2025-09-16 16:39:23 | INFO | Monitoring is not currently active
2025-09-16 16:39:23 | INFO | Disconnected from Pfeiffer device tpg366_dual_pressure
2025-09-16 16:39:23 | INFO | TPG366 pressure controller disconnected
2025-09-16 16:39:23 | INFO | Disconnected from chiller lauda_chiller
2025-09-16 16:39:23 | INFO | Chiller disconnected
2025-09-16 16:39:23 | SUCCESS | All devices disconnected


In [22]:
# Emergency stop and cleanup
print("🛑 Emergency Stop - Disconnecting all devices...")
controller.disconnect_all()
print("✅ All devices disconnected safely")
print("\nTo reconnect, run the connection cells again.")

2025-09-15 15:15:08 | INFO | Disconnecting all devices...
2025-09-15 15:15:08 | INFO | Monitoring is not currently active
2025-09-15 15:15:08 | INFO | Disconnected from Pfeiffer device tpg366_dual_pressure
2025-09-15 15:15:08 | INFO | TPG366 pressure controller disconnected
2025-09-15 15:15:08 | INFO | Disconnected from chiller lauda_chiller
2025-09-15 15:15:08 | INFO | Chiller disconnected
2025-09-15 15:15:08 | SUCCESS | All devices disconnected


🛑 Emergency Stop - Disconnecting all devices...
✅ All devices disconnected safely

To reconnect, run the connection cells again.


## Quick Reference

### Key Functions:
- **Connect devices**: Run cells in section 6
- **Start monitoring**: Use the interface in section 8 or call `controller.start_monitoring(interval)`
- **Stop monitoring**: Use the interface in section 8 or call `controller.stop_monitoring()`
- **View data**: Run `get_monitoring_summary()` and `plot_monitoring_data()`
- **Export data**: Run `export_monitoring_data()` or `save_monitoring_data_pickle()`
- **Emergency stop**: Run the last cell to disconnect everything

### Test Configuration:
- **A100 Pressure**: TPG366 Channel 1
- **A200 Pressure**: TPG366 Channel 2  
- **Chiller Temperature**: Current and set temperature monitoring
- **Data Logging**: All values logged simultaneously with timestamps

### Key Improvements from Original 008:
- Dual pressure sensor monitoring (A100 & A200) instead of single sensor
- Added chiller (Lauda) temperature monitoring and control
- Enhanced live plotting with temperature display
- Updated data export functions to include all three data streams
- Shared logging system for all devices (TPG366 + Chiller)
- Real-time visualization of pressure and temperature trends

### Device Configuration:
- Pressure Controller: TPG366 (standard address 10, channels 1 & 2)
- Chiller: Lauda (COM3, temperature monitoring and control)
- All devices use shared logger and thread synchronization

### Live Monitoring Features:
- **Pressure**: A100 and A200 on logarithmic scale (typical for vacuum measurements)
- **Temperature**: Current vs. set temperature comparison  
- **Real-time Updates**: Configurable interval from 0.1 to 30 seconds
- **Data Export**: CSV and pickle formats for analysis
- **Thread-Safe**: Proper synchronization for all device communications

# Saving Plot as html with holoviz

In [ ]:
import pandas as pd
import hvplot.pandas
import holoviews as hv
from bokeh.plotting import output_file, save
from bokeh.io import curdoc
import numpy as np

# Initialize HoloViews with Bokeh backend
hv.extension('bokeh')

def create_interactive_monitoring_plot(save_path="monitoring_data.html", downsample_factor=10):
    """Create an interactive HTML plot using hvPlot for monitoring data

    Args:
        save_path: Path to save HTML file
        downsample_factor: Reduce data points by this factor for performance (10 = every 10th point)
    """
    if not controller.monitoring_data:
        print("No monitoring data available. Start monitoring first.")
        return

    print(f"Processing {len(controller.monitoring_data)} data points...")

    # Convert monitoring data to pandas DataFrame with downsampling for performance
    data_records = []
    for i, cycle in enumerate(controller.monitoring_data):
        # Downsample for performance with large datasets
        if i % downsample_factor != 0:
            continue

        record = {
            'timestamp': cycle['cycle_timestamp'],
            'a100_pressure': cycle['a100_data']['pressure_value'] if cycle['a100_data'] and cycle['a100_data'].get('pressure_value') else np.nan,
            'a200_pressure': cycle['a200_data']['pressure_value'] if cycle['a200_data'] and cycle['a200_data'].get('pressure_value') else np.nan,
            'chiller_current_temp': cycle['chiller_data']['current_temperature'] if cycle['chiller_data'] and cycle['chiller_data'].get('current_temperature') else np.nan,
        }
        data_records.append(record)

    df = pd.DataFrame(data_records)
    print(f"DataFrame created with {len(df)} points (downsampled from {len(controller.monitoring_data)})")

    # Check data validity
    print(f"A100 pressure valid points: {df['a100_pressure'].notna().sum()}")
    print(f"A200 pressure valid points: {df['a200_pressure'].notna().sum()}")
    print(f"Temperature valid points: {df['chiller_current_temp'].notna().sum()}")

    if df.empty:
        print("No valid data to plot!")
        return None

    try:
        # Create pressure plot with logarithmic scale
        pressure_plot = df.hvplot.line(
            x='timestamp',
            y=['a100_pressure', 'a200_pressure'],
            logy=True,
            title='A100 & A200 Pressure Over Time (Log Scale)',
            ylabel='Pressure (log scale) // hpa',
            xlabel='',
            width=900,
            height=400,
            color=['blue', 'red'],
            line_width=1.5,
            alpha=0.8,
            tools=['hover', 'pan', 'wheel_zoom', 'box_zoom', 'reset', 'save'],
            hover_cols=['timestamp'],
            legend='top_right'
        ).opts(
            show_grid=True,
            toolbar='above'
        )

        # Create temperature plot
        temp_plot = df.hvplot.line(
            x='timestamp',
            y='chiller_current_temp',
            title='Chiller Temperature Over Time',
            ylabel='Temperature // degC',
            xlabel='Time',
            width=900,
            height=400,
            color='green',
            line_width=1.5,
            alpha=0.8,
            tools=['hover', 'pan', 'wheel_zoom', 'box_zoom', 'reset', 'save'],
            hover_cols=['timestamp']
        ).opts(
            show_grid=True,
            toolbar='above'
        )

        # Combine plots vertically with shared x-axis
        combined_plot = (pressure_plot + temp_plot).cols(1).opts(shared_axes=True)

        # Save the plot - try multiple methods
        success = False

        # Method 1: Direct hvplot save
        try:
            combined_plot.save(save_path)
            print(f"✅ Plot saved successfully: {save_path}")
            success = True
        except Exception as e:
            print(f"❌ hvplot.save() failed: {e}")

        # Method 2: HoloViews renderer
        if not success:
            try:
                renderer = hv.renderer('bokeh').instance(fig='html')
                renderer.save(combined_plot, save_path)
                print(f"✅ Plot saved via HoloViews renderer: {save_path}")
                success = True
            except Exception as e:
                print(f"❌ HoloViews renderer failed: {e}")

        # Method 3: Convert to Bokeh plot directly
        if not success:
            try:
                from bokeh.plotting import output_file, save
                bokeh_plot = hv.render(combined_plot, backend='bokeh')
                output_file(save_path)
                save(bokeh_plot)
                print(f"✅ Plot saved via Bokeh: {save_path}")
                success = True
            except Exception as e:
                print(f"❌ Bokeh save failed: {e}")

        if not success:
            print("❌ All save methods failed!")
            return None

        # Print statistics
        print(f"\n📊 Plot Statistics:")
        print(f"   Total original data points: {len(controller.monitoring_data)}")
        print(f"   Plotted data points: {len(df)} (downsampled by factor {downsample_factor})")

        valid_a100 = df['a100_pressure'].dropna()
        valid_a200 = df['a200_pressure'].dropna()
        valid_temp = df['chiller_current_temp'].dropna()

        if len(valid_a100) > 0:
            print(f"   A100 pressure range: {valid_a100.min():.2e} to {valid_a100.max():.2e}")
        if len(valid_a200) > 0:
            print(f"   A200 pressure range: {valid_a200.min():.2e} to {valid_a200.max():.2e}")
        if len(valid_temp) > 0:
            print(f"   Temperature range: {valid_temp.min():.2f}°C to {valid_temp.max():.2f}°C")

        return combined_plot

    except Exception as e:
        print(f"❌ Error creating plots: {e}")
        import traceback
        traceback.print_exc()
        return None

def create_bokeh_fallback_plot(save_path="monitoring_bokeh.html", downsample_factor=10):
    """Fallback: Create plot directly with Bokeh"""
    if not controller.monitoring_data:
        print("No monitoring data available.")
        return

    from bokeh.plotting import figure, save, output_file
    from bokeh.layouts import column
    from bokeh.models import DatetimeTickFormatter, HoverTool

    print(f"Creating Bokeh plot with {len(controller.monitoring_data)} data points...")

    # Extract and downsample data
    timestamps = []
    a100_pressure = []
    a200_pressure = []
    chiller_current_temp = []

    for i, cycle in enumerate(controller.monitoring_data):
        if i % downsample_factor != 0:
            continue

        timestamps.append(cycle['cycle_timestamp'])

        a100_val = cycle['a100_data']['pressure_value'] if cycle['a100_data'] and cycle['a100_data'].get('pressure_value') else None
        a200_val = cycle['a200_data']['pressure_value'] if cycle['a200_data'] and cycle['a200_data'].get('pressure_value') else None
        temp_val = cycle['chiller_data']['current_temperature'] if cycle['chiller_data'] and cycle['chiller_data'].get('current_temperature') else None

        a100_pressure.append(a100_val)
        a200_pressure.append(a200_val)
        chiller_current_temp.append(temp_val)

    print(f"Downsampled to {len(timestamps)} points")

    # Create pressure plot with hover tool
    hover1 = HoverTool(tooltips=[("Time", "@x{%H:%M:%S}"), ("Pressure", "@y{0.00e+0}")],
                       formatters={'@x': 'datetime'})

    p1 = figure(title="A100 & A200 Pressure Over Time (Log Scale)",
                x_axis_type='datetime', y_axis_type="log",
                width=900, height=400, tools=[hover1, 'pan', 'wheel_zoom', 'box_zoom', 'reset', 'save'])

    p1.line(timestamps, a100_pressure, legend_label="A100 Pressure", line_width=2, color='blue', alpha=0.8)
    p1.line(timestamps, a200_pressure, legend_label="A200 Pressure", line_width=2, color='red', alpha=0.8)
    p1.xaxis.formatter = DatetimeTickFormatter(hours="%H:%M:%S", minutes="%H:%M:%S")
    p1.legend.location = "top_right"
    p1.yaxis.axis_label = "Pressure (log scale)"

    # Create temperature plot with hover tool
    hover2 = HoverTool(tooltips=[("Time", "@x{%H:%M:%S}"), ("Temperature", "@y{0.0}°C")],
                       formatters={'@x': 'datetime'})

    p2 = figure(title="Chiller Temperature Over Time",
                x_axis_type='datetime', width=900, height=400,
                x_range=p1.x_range,  # Link x-axes
                tools=[hover2, 'pan', 'wheel_zoom', 'box_zoom', 'reset', 'save'])

    p2.line(timestamps, chiller_current_temp, legend_label="Current Temperature", line_width=2, color='green', alpha=0.8)
    p2.xaxis.formatter = DatetimeTickFormatter(hours="%H:%M:%S", minutes="%H:%M:%S")
    p2.legend.location = "top_right"
    p2.yaxis.axis_label = "Temperature (°C)"
    p2.xaxis.axis_label = "Time"

    # Combine plots
    layout = column(p1, p2)

    # Save
    output_file(save_path)
    save(layout)
    print(f"✅ Bokeh plot saved as: {save_path}")

    return layout

# Usage examples:
#print("🚀 Interactive plotting functions ready!")
#print("Usage:")
#print("  create_interactive_monitoring_plot('my_plot.html', downsample_factor=10)")
#print("  create_bokeh_fallback_plot('my_plot_bokeh.html', downsample_factor=10)")
#print(f"📊 Your dataset has {len(controller.monitoring_data) if 'controller' in globals() else 'N/A'} data points")

In [ ]:
# Example usage:
plot = create_interactive_monitoring_plot("my_monitoring_data.html")
#create_bokeh_fallback_plot("test_bokeh.html", downsample_factor=10)

# Test holoviz with Box desscribtion

In [32]:
import pandas as pd
import hvplot.pandas
import holoviews as hv
from bokeh.plotting import output_file, save
from bokeh.io import curdoc
import numpy as np

# Initialize HoloViews with Bokeh backend
hv.extension('bokeh')

def create_interactive_monitoring_plot(save_path="monitoring_data.html", downsample_factor=10):
    """Create an interactive HTML plot using hvPlot for monitoring data
    
    Args:
        save_path: Path to save HTML file
        downsample_factor: Reduce data points by this factor for performance (10 = every 10th point)
    """
    if not controller.monitoring_data:
        print("No monitoring data available. Start monitoring first.")
        return
    
    print(f"Processing {len(controller.monitoring_data)} data points...")
    
    # Convert monitoring data to pandas DataFrame with downsampling for performance
    data_records = []
    for i, cycle in enumerate(controller.monitoring_data):
        # Downsample for performance with large datasets
        if i % downsample_factor != 0:
            continue
            
        record = {
            'timestamp': cycle['cycle_timestamp'],
            'a100_pressure': cycle['a100_data']['pressure_value'] if cycle['a100_data'] and cycle['a100_data'].get('pressure_value') else np.nan,
            'a200_pressure': cycle['a200_data']['pressure_value'] if cycle['a200_data'] and cycle['a200_data'].get('pressure_value') else np.nan,
            'chiller_current_temp': cycle['chiller_data']['current_temperature'] if cycle['chiller_data'] and cycle['chiller_data'].get('current_temperature') else np.nan,
        }
        data_records.append(record)
    
    df = pd.DataFrame(data_records)
    print(f"DataFrame created with {len(df)} points (downsampled from {len(controller.monitoring_data)})")
    
    # Check data validity
    print(f"A100 pressure valid points: {df['a100_pressure'].notna().sum()}")
    print(f"A200 pressure valid points: {df['a200_pressure'].notna().sum()}")
    print(f"Temperature valid points: {df['chiller_current_temp'].notna().sum()}")
    
    if df.empty:
        print("No valid data to plot!")
        return None
    
    try:
        # Create pressure plot with logarithmic scale
        pressure_plot = df.hvplot.line(
            x='timestamp', 
            y=['a100_pressure', 'a200_pressure'],
            logy=True,
            title='A100 & A200 Pressure Over Time (Log Scale)',
            ylabel='Pressure (log scale)',
            width=900,
            height=400,
            color=['blue', 'red'],
            line_width=1.5,
            alpha=0.8,
            tools=['hover', 'pan', 'wheel_zoom', 'box_zoom', 'reset', 'save'],
            hover_cols=['timestamp'],
            legend='top_right'
        ).opts(
            show_grid=True,
            toolbar='above'
        )
        
        # Create temperature plot
        temp_plot = df.hvplot.line(
            x='timestamp',
            y='chiller_current_temp',
            title='Chiller Temperature Over Time',
            ylabel='Temperature (°C)',
            xlabel='Time',
            width=900,
            height=400,
            color='green',
            line_width=1.5,
            alpha=0.8,
            tools=['hover', 'pan', 'wheel_zoom', 'box_zoom', 'reset', 'save'],
            hover_cols=['timestamp']
        ).opts(
            show_grid=True,
            toolbar='above'
        )
        
        # Combine plots vertically with shared x-axis
        combined_plot = (pressure_plot + temp_plot).cols(1).opts(shared_axes=True)
        
        # Save the plot - try multiple methods
        success = False
        
        # Method 1: Direct hvplot save
        try:
            combined_plot.save(save_path)
            print(f"✅ Plot saved successfully: {save_path}")
            success = True
        except Exception as e:
            print(f"❌ hvplot.save() failed: {e}")
        
        # Method 2: HoloViews renderer
        if not success:
            try:
                renderer = hv.renderer('bokeh').instance(fig='html')
                renderer.save(combined_plot, save_path)
                print(f"✅ Plot saved via HoloViews renderer: {save_path}")
                success = True
            except Exception as e:
                print(f"❌ HoloViews renderer failed: {e}")
        
        # Method 3: Convert to Bokeh plot directly
        if not success:
            try:
                from bokeh.plotting import output_file, save
                bokeh_plot = hv.render(combined_plot, backend='bokeh')
                output_file(save_path)
                save(bokeh_plot)
                print(f"✅ Plot saved via Bokeh: {save_path}")
                success = True
            except Exception as e:
                print(f"❌ Bokeh save failed: {e}")
        
        if not success:
            print("❌ All save methods failed!")
            return None
        
        # Print statistics
        print(f"\n📊 Plot Statistics:")
        print(f"   Total original data points: {len(controller.monitoring_data)}")
        print(f"   Plotted data points: {len(df)} (downsampled by factor {downsample_factor})")
        
        valid_a100 = df['a100_pressure'].dropna()
        valid_a200 = df['a200_pressure'].dropna()
        valid_temp = df['chiller_current_temp'].dropna()

        if len(valid_a100) > 0:
            print(f"   A100 pressure range: {valid_a100.min():.2e} to {valid_a100.max():.2e}")
        if len(valid_a200) > 0:
            print(f"   A200 pressure range: {valid_a200.min():.2e} to {valid_a200.max():.2e}")
        if len(valid_temp) > 0:
            print(f"   Temperature range: {valid_temp.min():.2f}°C to {valid_temp.max():.2f}°C")
        
        return combined_plot
        
    except Exception as e:
        print(f"❌ Error creating plots: {e}")
        import traceback
        traceback.print_exc()
        return None

def create_bokeh_fallback_plot(save_path="monitoring_bokeh.html", downsample_factor=10):
    """Fallback: Create plot directly with Bokeh"""
    if not controller.monitoring_data:
        print("No monitoring data available.")
        return
    
    from bokeh.plotting import figure, save, output_file
    from bokeh.layouts import column
    from bokeh.models import DatetimeTickFormatter, HoverTool
    
    print(f"Creating Bokeh plot with {len(controller.monitoring_data)} data points...")
    
    # Extract and downsample data
    timestamps = []
    a100_pressure = []
    a200_pressure = []
    chiller_current_temp = []

    for i, cycle in enumerate(controller.monitoring_data):
        if i % downsample_factor != 0:
            continue
            
        timestamps.append(cycle['cycle_timestamp'])
        
        a100_val = cycle['a100_data']['pressure_value'] if cycle['a100_data'] and cycle['a100_data'].get('pressure_value') else None
        a200_val = cycle['a200_data']['pressure_value'] if cycle['a200_data'] and cycle['a200_data'].get('pressure_value') else None
        temp_val = cycle['chiller_data']['current_temperature'] if cycle['chiller_data'] and cycle['chiller_data'].get('current_temperature') else None
        
        a100_pressure.append(a100_val)
        a200_pressure.append(a200_val)
        chiller_current_temp.append(temp_val)
    
    print(f"Downsampled to {len(timestamps)} points")
    
    # Create pressure plot with hover tool
    hover1 = HoverTool(tooltips=[("Time", "@x{%H:%M:%S}"), ("Pressure", "@y{0.00e+0}")],
                       formatters={'@x': 'datetime'})
    
    p1 = figure(title="A100 & A200 Pressure Over Time (Log Scale)", 
                x_axis_type='datetime', y_axis_type="log", 
                width=900, height=400, tools=[hover1, 'pan', 'wheel_zoom', 'box_zoom', 'reset', 'save'])
    
    p1.line(timestamps, a100_pressure, legend_label="A100 Pressure", line_width=2, color='blue', alpha=0.8)
    p1.line(timestamps, a200_pressure, legend_label="A200 Pressure", line_width=2, color='red', alpha=0.8)
    p1.xaxis.formatter = DatetimeTickFormatter(hours="%H:%M:%S", minutes="%H:%M:%S")
    p1.legend.location = "top_right"
    p1.yaxis.axis_label = "Pressure (log scale)"
    
    # Create temperature plot with hover tool
    hover2 = HoverTool(tooltips=[("Time", "@x{%H:%M:%S}"), ("Temperature", "@y{0.0}°C")],
                       formatters={'@x': 'datetime'})
    
    p2 = figure(title="Chiller Temperature Over Time", 
                x_axis_type='datetime', width=900, height=400, 
                x_range=p1.x_range,  # Link x-axes
                tools=[hover2, 'pan', 'wheel_zoom', 'box_zoom', 'reset', 'save'])
    
    p2.circle(timestamps, chiller_current_temp, legend_label="Current Temperature", size=6, color='green', alpha=0.7)
    p2.xaxis.formatter = DatetimeTickFormatter(hours="%H:%M:%S", minutes="%H:%M:%S")
    p2.legend.location = "top_right"
    p2.yaxis.axis_label = "Temperature (°C)"
    p2.xaxis.axis_label = "Time"
    
    # Combine plots
    layout = column(p1, p2)
    
    # Save
    output_file(save_path)
    save(layout)
    print(f"✅ Bokeh plot saved as: {save_path}")
    
    return layout

# Usage examples:
#print("🚀 Interactive plotting functions ready!")
#print("Usage:")
#print("  create_interactive_monitoring_plot('my_plot.html', downsample_factor=10, test_info='Test conditions...')")
#print("  create_bokeh_fallback_plot('my_plot_bokeh.html', downsample_factor=10, test_info='Test conditions...')")
#print("Example:")
#print("  test_info = '''")
#print("  Test Date: 2025-09-10")
#print("  Sample: XYZ-123")
#print("  Conditions: 25°C ambient")
#print("  Notes: Baseline measurement")
#print("  '''")
#print("  create_interactive_monitoring_plot('test.html', test_info=test_info)")
#print(f"📊 Your dataset has {len(controller.monitoring_data) if 'controller' in globals() else 'N/A'} data points")

In [33]:
# Define your test information
test_info = """
Test Date: 2025-09-12
Operator: Niclas Przibylla
Temperature Set Chiller: 16°C
Notes:
- Testing water circuit with only A200 L running & connected to water circuit
"""

# Create the plot with info box
create_interactive_monitoring_plot("009_A100_only_withN2.html", downsample_factor=10)

# Or with the Bokeh fallback
#create_bokeh_fallback_plot("my_test_bokeh.html", downsample_factor=10, test_info=test_info)

Processing 9346 data points...
DataFrame created with 935 points (downsampled from 9346)
A100 pressure valid points: 935
A200 pressure valid points: 935
Temperature valid points: 935
❌ hvplot.save() failed: 'Layout' object has no attribute save.
✅ Plot saved via HoloViews renderer: 009_A100_only_withN2.html

📊 Plot Statistics:
   Total original data points: 9346
   Plotted data points: 935 (downsampled by factor 10)
   A100 pressure range: 1.08e-03 to 1.02e+03
   A200 pressure range: 1.00e+03 to 1.01e+03
   Temperature range: 16.15°C to 17.46°C


:Layout
   .NdOverlay.I :NdOverlay   [Variable]
      :Curve   [timestamp]   (value)
   .Curve.I     :Curve   [timestamp]   (chiller_current_temp)